<img src="data/images/quantmetry.png" width=400>
<center><h1>TALAGRAND</h1></center>
<center><h2>Traitement Automatique du LAngage au service du GRANd Débat National</h2></center>


Ce notebook met à disposition des citoyens des outils d'Intelligence Artificielle pour appréhender le nombre important de contributions au Grand Débat.

Mots clés : **NLP, TAL, LDA, Embeddings, TextRank**

Pour plus d'informations, se référer à cet [article](quantmetry.com/grand-debat).

Auteurs : [Antoine SIMOULIN](asimoulin@quantmetry.com), [Antoine ISNARDY](aisnardy@quantmetry.com), [Thibaud REAL DEL SARTE](trealdelsarte@quantmetry.com)

**DISCLAIMER** : Quantmetry et les auteurs de ce notebook ne peuvent être tenus responsables de l'interprétation des réponses aux questions posées lors du Grand Débat. Le présent notebook fournit une grille d'analyse desdites réponses, non une interprétation.

In [ ]:
import warnings; warnings.simplefilter('ignore')

# Téléchargement des données
-  Récupérer la liste des thèmes en analysant le site du Grand Débat
-  Sélectionner un thème en modifiant la variable `selected_theme`
-  Télécharger ensuite le fichier json le plus récent associé à un thème depuis datagouv.fr

In [ ]:
from grand_debat.downloading import download_data, get_themes

In [ ]:
themes = get_themes()
list(themes.keys())

In [ ]:
# Choisir le numéro associé au thème que l'on souhaite étudier
selected_theme = 4

In [ ]:
download_data(themes, selected_theme)

# Chargement des données
-  Charger le fichier json précédemment téléchargé
-  Afficher les différentes questions associées au thème sélectionné
-  Sélectionner une question à étudier (variable `selected_question`)
-  Afficher le nombre de réponses existantes correspondant à cette question

In [ ]:
from grand_debat.loading import load_data, load_question, get_path

In [ ]:
path = get_path(themes, selected_theme)
data_theme_json = load_data(path)

In [ ]:
data_theme_json

In [ ]:
questions_id = [r['questionId'] for x in data_theme_json for r in x['responses']]
questions_id = list(set(questions_id))
questions_title = [r['questionTitle'] for x in data_theme_json for r in x['responses']]
questions_title = list(set(questions_title))
for id, title in zip(questions_id, questions_title):
    print("ID : {} - Question: {}".format(id, title))
    print()

In [ ]:
# Indiquer ici l'identifiant associé à la question étudiée
selected_question = 'UXVlc3Rpb246MTU5'

In [ ]:
selected_question = str(selected_question)
data_theme_response_dict = load_question(questions_id, data_theme_json, selected_question)
print("Nombre de réponses analysées :", len(data_theme_response_dict[selected_question]))

## <h2>Détection de thèmes</h2>
-  Détecter les thèmes principaux associés à la question sélectionnée en utilisant l'algorithme **LDA** (Latent Dirichlet Allocation)
-  Le resultat de l'analyse se trouve dans le fichier pyLDAVIS_tf.html

In [ ]:
from grand_debat.theme_detection import get_themes
tf_vectorizer, lda_tf = get_themes(data_theme_response_dict, selected_question)
print("Les différents thèmes associés à la question se situent dans le fichier pyLDAVIS_tf.html")

## <h2>Résumé de chaque thème</h2>
-  Résumer un grand thème en utilisant TextRank, technique de résumé extractive
-  Spécifier le nombre de phrases que l'on souhaite pour former le résumé dans la variable `sn`
-  Les résultats se situent dans le fichier demo.docx

In [ ]:
from grand_debat.text_summarization import prepare_data, apply_page_rank_algorithm, create_titles
from grand_debat.vectorization import sentence_to_vec
from grand_debat.clusterisation import apply_clusterisation

In [ ]:
#import nltk
#nltk.download('punkt')

Transformer le document texte en un ensemble de phrases dont la typographie est standardisée

In [ ]:
clean_sentences = prepare_data(data_theme_response_dict, selected_question)

Récupérer la phrase la plus caractéristique de chaque thème

In [ ]:
sentences_mixture_topics, title_sentences, titles = create_titles(clean_sentences, tf_vectorizer, lda_tf)

Associer à chaque phrase du corpus son thème (issu de **LDA**) le plus carctéristique

In [ ]:
sentences_paragraph = apply_clusterisation(sentences_mixture_topics)

Représenter chaque phrase sous la forme d'un vecteur dense. Ces vecteurs sont obtenus à partir d'embeddings entraînés à partir de l'ensemble des contributions au Grand Débat

In [ ]:
sentence_vectors, word_embeddings = sentence_to_vec(clean_sentences, file_model='word2vec.38k.150d.txt')

On applique alors l'algorithme **TextRank** à l'ensemble des phrases d'un thème pour en extraire les phrases carctéristiques

In [ ]:
# Indiquer le nombre de phrases qui formeront le résumé
sn = 10
apply_page_rank_algorithm(clean_sentences, sentences_paragraph, word_embeddings, sn)

## Export de la synthèse sous la forme d'un document word
-  Créer le document final contenant les résumés pour chaque thème de la question.
-  Le processus prend environ 20 minutes sur une machine équipée d'un intel core i5.

In [ ]:
from grand_debat.saving import create_final_doc, get_theme, get_question

In [ ]:
theme_sentence = get_theme(themes, selected_theme)
question_sentence = get_question(questions_id, questions_title, selected_question)
create_final_doc(title_sentences, titles, clean_sentences,
                 sentences_paragraph, word_embeddings, theme_sentence, question_sentence)